## Deliverable 2. Create a Customer Travel Destinations Map.

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt
import numpy as np

# Configure API keys
from config import weather_api_key
#from config import j_key

In [6]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rio Grande,BR,-32.0350,-52.0986,70.00,95,100,8.32,light rain
1,1,Rikitea,PF,-23.1203,-134.9692,78.73,73,14,18.19,light rain
2,2,Punta Arenas,CL,-53.1500,-70.9167,48.20,71,75,8.05,broken clouds
3,3,Vangaindrano,MG,-23.3500,47.6000,71.24,95,27,3.42,scattered clouds
4,4,Varadero,CU,23.1536,-81.2514,77.00,61,20,12.66,few clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 90


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rio Grande,BR,-32.0350,-52.0986,70.00,95,100,8.32,light rain
1,1,Rikitea,PF,-23.1203,-134.9692,78.73,73,14,18.19,light rain
3,3,Vangaindrano,MG,-23.3500,47.6000,71.24,95,27,3.42,scattered clouds
4,4,Varadero,CU,23.1536,-81.2514,77.00,61,20,12.66,few clouds
5,5,Port Alfred,ZA,-33.5906,26.8910,68.52,84,83,21.85,broken clouds
7,7,Bambous Virieux,MU,-20.3428,57.7575,75.99,88,20,6.91,few clouds
9,9,Puerto Ayora,EC,-0.7393,-90.3518,79.00,86,49,5.01,scattered clouds
11,11,Isangel,VU,-19.5500,169.2667,84.20,79,75,4.61,broken clouds
16,16,Odweyne,SO,9.4092,45.0640,66.40,42,41,7.49,scattered clouds
19,19,Avera,US,33.1940,-82.5271,66.99,94,90,4.63,fog


In [11]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             208
City                   208
Country                208
Lat                    208
Lng                    208
Max Temp               208
Humidity               208
Cloudiness             208
Wind Speed             208
Current Description    208
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [13]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rio Grande,BR,70.00,light rain,-32.0350,-52.0986,
1,Rikitea,PF,78.73,light rain,-23.1203,-134.9692,
3,Vangaindrano,MG,71.24,scattered clouds,-23.3500,47.6000,
4,Varadero,CU,77.00,few clouds,23.1536,-81.2514,
5,Port Alfred,ZA,68.52,broken clouds,-33.5906,26.8910,
7,Bambous Virieux,MU,75.99,few clouds,-20.3428,57.7575,
9,Puerto Ayora,EC,79.00,scattered clouds,-0.7393,-90.3518,
11,Isangel,VU,84.20,broken clouds,-19.5500,169.2667,
16,Odweyne,SO,66.40,scattered clouds,9.4092,45.0640,
19,Avera,US,66.99,fog,33.1940,-82.5271,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": j_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


hotel_df.head(10)    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rio Grande,BR,70.00,light rain,-32.0350,-52.0986,Hotel Atlântico Rio Grande
1,Rikitea,PF,78.73,light rain,-23.1203,-134.9692,Pension Maro'i
3,Vangaindrano,MG,71.24,scattered clouds,-23.3500,47.6000,Hotel Fanilo
4,Varadero,CU,77.00,few clouds,23.1536,-81.2514,Hotel Tuxpan Varadero
5,Port Alfred,ZA,68.52,broken clouds,-33.5906,26.8910,The Halyards Hotel
7,Bambous Virieux,MU,75.99,few clouds,-20.3428,57.7575,Casa Tia Villa
9,Puerto Ayora,EC,79.00,scattered clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
11,Isangel,VU,84.20,broken clouds,-19.5500,169.2667,Volcano Sea View Tree House
16,Odweyne,SO,66.40,scattered clouds,9.4092,45.0640,Eng Hiirey House
19,Avera,US,66.99,fog,33.1940,-82.5271,


In [17]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace("", np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rio Grande,BR,70.00,light rain,-32.0350,-52.0986,Hotel Atlântico Rio Grande
1,Rikitea,PF,78.73,light rain,-23.1203,-134.9692,Pension Maro'i
3,Vangaindrano,MG,71.24,scattered clouds,-23.3500,47.6000,Hotel Fanilo
4,Varadero,CU,77.00,few clouds,23.1536,-81.2514,Hotel Tuxpan Varadero
5,Port Alfred,ZA,68.52,broken clouds,-33.5906,26.8910,The Halyards Hotel


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file)

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map.
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))